# Practical Quantization in PyTorch

https://pytorch.org/blog/quantization-in-practice/#fundamentals-of-quantization

`Quantization` is a way to make your neural net architecture to run faster and with lower memory requirements. There are various different approaches to `quantize` a model.

## Fundamentals of Quantization
`Quantization` has its roots in information compression or as in deep learning, the `numerical precision` of its `weights`/`activation`.

`Overparameterized Deep Neural Net` have more degrees of freedom and this makes them good for Information Compression. When we `quantize` a model, two things happen - the model gets `smaller` and `runs` with better `efficiency`. Hardware vendors explicitly allow for faster processing of `8-bit` data resulting `higher` throughput. A smaller model has `lower memory footprint` and `power consumption`, crucial for deployment at the `edge`.

### Mapping Function
The mapping function is a fn that maps values from `floating-point` to integer space. A commonly used mapping function is <b>linear transformation</b> given by <img src="https://latex.codecogs.com/gif.latex?Q(r)%20=%20round(r/S%20+%20Z)"/> 

where `r` is the input and `S, Z` are <b>quantization parameters</b>.

To convert to floating point space, the inverse function is given by <img src="https://latex.codecogs.com/gif.latex?\tilde%20r%20=%20(Q(r)%20-%20Z)%20\cdot%20S"/>

<img src="https://latex.codecogs.com/gif.latex?\tilde%20r%20\neq%20r"/>, and their difference constitutes the `quantization error`.

### Quantization Parameters
The mapping function is parameterized by the `scaling factor S` and `zero point Z`. `S` is simply the ratio of the input range to the output range <img src="https://latex.codecogs.com/gif.latex?S%20=%20\frac{\beta%20-%20\alpha}{\beta_q%20-%20\alpha_q}"/>

where [alpha, Beta] is the `clipping range` of the input i.e the boundaries of permissible inputs. <img src="https://latex.codecogs.com/gif.latex?\alpha_q,%20\beta_q"/> is the range in quantized output space that it is mapped to. For `8bit` quantization, the output range <img src="https://latex.codecogs.com/gif.latex?\beta_q%20-%20\alpha_q%20%3C=%20(2^8%20-%201)"/>

`Z` acts as a `bias` to ensure that a 0 in the input space maps perfectly to a `0` in the quantized space. <img src="https://latex.codecogs.com/gif.latex?Z%20=%20-(\frac{\alpha}{S}%20-%20\alpha_q)"/>

### Calibration
The process of choosing the input clipping range is known `calibration`. The simplest technique is to record the running minimum and maximum values and assign them to `alpha` and `beta`. `TensorRT` uses entropy minimization, mean-square-error minimization or percentiles of the input range.

`Observer` is module which collects statistics on the input values and calculate the qparams `S, Z`. Different `calibration` schemes result in different quantized outputs, and its best to verify which scheme works best for an application and an architecture

In [4]:
import torch
from torch.quantization.observer import MinMaxObserver, MovingAverageMinMaxObserver, HistogramObserver
C, L = 3, 4

normal = torch.distributions.normal.Normal(0, 1)
inputs = [normal.sample((C, L)), normal.sample((C, L))]
print(inputs)

[tensor([[-0.2156,  1.1460, -0.2014, -0.4275],
        [ 0.8727,  1.1238, -0.9292,  0.7353],
        [-0.3097, -0.7722,  1.4937,  0.2940]]), tensor([[ 1.7389,  0.6298, -2.2632,  0.0851],
        [-0.1918, -1.0293,  0.2904, -2.2416],
        [ 1.1828, -0.6239,  1.5094,  0.9856]])]


In [5]:
observers = [MinMaxObserver(), MovingAverageMinMaxObserver(), HistogramObserver()]
for obs in observers:
    for x in inputs:
        obs(x)
    print(obs.__class__.__name__, obs.calculate_qparams())

MinMaxObserver (tensor([0.0157]), tensor([144], dtype=torch.int32))
MovingAverageMinMaxObserver (tensor([0.0096]), tensor([99], dtype=torch.int32))
HistogramObserver (tensor([0.0107]), tensor([91], dtype=torch.int32))


### Affine and Symmetric Quantization Schemes
`Affine or asymmetric quantization` scheme assign the input range to the `min` and `max` observed values. Affine schemes genrally offer tighter clipping ranges and are useful for quantizing non-negative activations (you don't need the input range to contain negative values.